<a href="https://www.kaggle.com/code/emmermarcell/training-an-open-book-model?scriptVersionId=154269568" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Training a model on the open book dataset
The second notebook of our homework contains the training process of the 'microsoft/deberta-v3-large' LLM on the open book database we created in the [context-creation][1] notebook. The entire notebook is heavily inspired from the popular competition notebook of the [Kaggle - LLM Science Exam][2], called [How To Train Open Book Model - Part 1][3]. There, [Chris Deotte][4] used a dataset containig 65k question and answer pairs with context compiled from different notebooks using e.g. gpt-3.5 turbo for data augmentation. This augmentation step was left out in our solution due to time constraints but it's an obvious choice of improvement in the future.

We use the 'microsoft/deberta-v3-large' on the openbook data simply because it turned out to be one of the most successful model in terms of highest [notebook scores][5] on kaggle


[1]: https://www.kaggle.com/code/emmermarcell/context-creation
[2]: https://www.kaggle.com/competitions/kaggle-llm-science-exam
[3]: https://www.kaggle.com/code/cdeotte/how-to-train-open-book-model-part-1
[4]: https://www.kaggle.com/cdeotte
[5]: https://www.kaggle.com/competitions/kaggle-llm-science-exam/models

In [1]:
import os

# Set CUDA visible devices to GPU 0 and 1 (The 2xT4 GPUs that Kaggle provides)
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"

from typing import Optional, Union
import pandas as pd, numpy as np, torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from datasets import Dataset, load_metric
from dataclasses import dataclass
import transformers
from transformers import AutoTokenizer, AutoModelForMultipleChoice, EarlyStoppingCallback, \
                         TrainingArguments, Trainer, set_seed
from transformers.tokenization_utils_base import PreTrainedTokenizerBase, PaddingStrategy

# Random seed
seed = 42
set_seed(seed)
# Define constants
VER=11
# Number of layers to freeze, DeBERTa has a total number of 24 layers
FREEZE_LAYERS = 18
# Boolean to freeze embeddings
FREEZE_EMBEDDINGS = True
# Length of context + question + answers
MAX_INPUT = 256
# The Hugging Face model we're using
MODEL = 'microsoft/deberta-v3-large'

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
# Import Weights & Biases library
import wandb

wandb.login()


%env WANDB_PROJECT=llm_science_exam_open_book_approach
%env WANDB_LOG_MODEL=true

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


env: WANDB_PROJECT=llm_science_exam_open_book_approach
env: WANDB_LOG_MODEL=true


# Load in the Input data

First we load in  the csv file generated in the [context-creation][1] notebook and split it into a training, validation, and a test set.

[1]: https://www.kaggle.com/code/emmermarcell/context-creation

In [3]:
# Read validation data from a CSV file
qna_df = pd.read_csv('/kaggle/input/context-creation/openbook-qna-data.csv')
print('Validation data size:', qna_df.shape )

qna_df.head()

Validation data size: (200, 8)


,prompt,A,B,C,D,E,answer,context
0,Which of the following statements accurately d...,MOND is a theory that reduces the observed mis...,MOND is a theory that increases the discrepanc...,MOND is a theory that explains the missing bar...,MOND is a theory that reduces the discrepancy ...,MOND is a theory that eliminates the observed ...,D,"In cosmology, the missing baryon problem is an..."
1,Which of the following is an accurate definiti...,Dynamic scaling refers to the evolution of sel...,Dynamic scaling refers to the non-evolution of...,Dynamic scaling refers to the evolution of sel...,Dynamic scaling refers to the non-evolution of...,Dynamic scaling refers to the evolution of sel...,A,Dynamic scaling (sometimes known as Family-Vic...
2,Which of the following statements accurately d...,The triskeles symbol was reconstructed as a fe...,The triskeles symbol is a representation of th...,The triskeles symbol is a representation of a ...,The triskeles symbol represents three interloc...,The triskeles symbol is a representation of th...,A,thumb|Neolithic triple spiral symbol A triskel...
3,What is the significance of regularization in ...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,C,"In physics, especially quantum field theory, r..."
4,Which of the following statements accurately d...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,D,Kinematic diffraction is the approach to study...


In [4]:
# train validation test split
train_df, temp_df = train_test_split(qna_df, test_size=0.5, random_state=seed)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=seed)

# Data Loader
We use a custom class for tokenizing the input data with dynamic padding in order to use it as an input to the ['microsoft/deberta-v3-large'][1] model.

The implementation of the preprocess function and the DataCollatorForMultipleChoice class is not ours, it's from Radek's notebook [here][2] with modifications to the tokenization process from Chris Deotte's notebook [here][3].

[1]: https://huggingface.co/microsoft/deberta-v3-large
[2]: https://www.kaggle.com/code/radek1/new-dataset-deberta-v3-large-training
[3]: https://www.kaggle.com/code/cdeotte/how-to-train-open-book-model-part-1

In [5]:
# Mapping options to indices
option_to_index = {option: idx for idx, option in enumerate('ABCDE')}
index_to_option = {v: k for k, v in option_to_index.items()}

def preprocess(example):
    # Repeat the prompt for all five choices
    first_sentence = [example['prompt']] * 5
    # Extract sentences corresponding to choices 'A' to 'E'
    second_sentences = [example[option] for option in 'ABCDE']
    # Tokenize the sentences using the provided tokenizer (not shown in this snippet)
    tokenized_example = tokenizer(first_sentence, second_sentences, truncation=False)
    # Assign the index corresponding to the correct answer as the label
    tokenized_example['label'] = option_to_index[example['answer']]

    return tokenized_example

@dataclass
class DataCollatorForMultipleChoice:
    tokenizer: PreTrainedTokenizerBase
    padding: Union[bool, str, PaddingStrategy] = True
    max_length: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None

    def __call__(self, features):
        # Determine the label name ('label' or 'labels')
        label_name = 'label' if 'label' in features[0].keys() else 'labels'
        # Extract labels from each example and remove the corresponding key
        labels = [feature.pop(label_name) for feature in features]
        # Compute batch size and number of choices
        batch_size = len(features)
        num_choices = len(features[0]['input_ids'])
        # Restructure features into a list of dictionaries for each choice
        flattened_features = [
            [{k: v[i] for k, v in feature.items()} for i in range(num_choices)] for feature in features
        ]
        # Flatten the list of dictionaries
        flattened_features = sum(flattened_features, [])

        # Tokenize and pad the examples into a batch
        batch = self.tokenizer.pad(
            flattened_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors='pt',
        )
        # Reshape the batch to have dimensions (batch_size, num_choices, -1)
        batch = {k: v.view(batch_size, num_choices, -1) for k, v in batch.items()}
        # Add labels to the batch as a PyTorch tensor
        batch['labels'] = torch.tensor(labels, dtype=torch.int64)
        # Return the formatted batch
        return batch

In [6]:
# Create tokenizer and datasets
tokenizer = AutoTokenizer.from_pretrained(MODEL)

train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)
test_dataset = Dataset.from_pandas(test_df)

train_dataset = train_dataset.remove_columns(["__index_level_0__"])

train_dataset

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/opt/conda/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:470: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Dataset({
    features: ['prompt', 'A', 'B', 'C', 'D', 'E', 'answer', 'context'],
    num_rows: 100
})

In [7]:
# Tokenize datasets
tokenized_train_dataset = train_dataset.map(preprocess, remove_columns=['prompt', 'context', 'A', 'B', 'C', 'D', 'E', 'answer'])
tokenized_val_dataset = val_dataset.map(preprocess, remove_columns=['prompt', 'context', 'A', 'B', 'C', 'D', 'E', 'answer'])
# We do not remove the answer column from the test dataset for evaluation
tokenized_test_dataset = test_dataset.map(preprocess, remove_columns=['prompt', 'context', 'A', 'B', 'C', 'D', 'E'])

tokenized_train_dataset

  0%|          | 0/100 [00:00<?, ?ex/s]

  0%|          | 0/50 [00:00<?, ?ex/s]

  0%|          | 0/50 [00:00<?, ?ex/s]

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'label'],
    num_rows: 100
})

# Building the model

In [8]:
def model_init():
    # Loading in the deberta model
    model = AutoModelForMultipleChoice.from_pretrained(MODEL)
    # We freeze the first 18 layers of the model for faster training time. 
    # However, his is compensated by lower valuation accuracy.
    if FREEZE_EMBEDDINGS:
        print('Freezing embeddings.')
        for param in model.deberta.embeddings.parameters():
            param.requires_grad = False
    if FREEZE_LAYERS>0:
        print(f'Freezing {FREEZE_LAYERS} layers.')
        for layer in model.deberta.encoder.layer[:FREEZE_LAYERS]:
            for param in layer.parameters():
                param.requires_grad = False
    
    return model

# Exploring Hyperparameter Combinations With Sweeps

In [9]:
# method and metric
sweep_config = {
    'method': 'bayes',
    'metric': {
        'name': 'map@3',
        'goal': 'maximize'
    },
}


# hyperparameters
parameters_dict = {
    'learning_rate': {
      'min': 1e-6,
      'max': 1e-4
    },
   'epochs':{
      'values': [20, 30, 50]
   },
    'weight_decay': {
      'values': [0.0, 0.01, 0.02, 0.03, 0.04, 0.05]
    },
    'warmup_ratio': {
      'values': [0.0, 0.05, 0.1, 0.15, 0.2]
    },
    'gradient_accumulation_steps': {
      'values': [2, 4, 8, 16]
    },
    'early_stopping_patience': {
      'values': [5, 10]
    },
}


sweep_config['parameters'] = parameters_dict

In [10]:
sweep_id = wandb.sweep(sweep_config, project='llm_science_exam_open_book_approach')

Create sweep with ID: qhm713tb
Sweep URL: https://wandb.ai/import_this/llm_science_exam_open_book_approach/sweeps/qhm713tb


# MAP@3 Metric
The competition metric is MAP@3 therefore we will make a custom code to add to Hugging Face's trainer. Discussion [here][1]

[1]: https://www.kaggle.com/competitions/kaggle-llm-science-exam/discussion/435602

In [11]:
def map_at_3(predictions, labels):
    map_sum = 0
    pred = np.argsort(-1*np.array(predictions),axis=1)[:,:3]
    for x,y in zip(pred,labels):
        z = [1/i if y==j else 0 for i,j in zip([1,2,3],x)]
        map_sum += np.sum(z)
    return map_sum / len(predictions)

# Define metrics computation function for Hugging Face Trainer
def compute_metrics(p):
    # computing the predictions and the labels
    predictions, labels = p.predictions, p.label_ids

    # Log multiple metrics: map@3 and accuracy
    return {"map@3": map_at_3(predictions.tolist(), labels.tolist()),
            "accuracy": accuracy_score(labels, predictions.argmax(axis=1))}

# Compute Validation Score

In [12]:
# https://www.kaggle.com/code/philippsinger/h2ogpt-perplexity-ranking
import numpy as np
def precision_at_k(r, k):
    """Precision at k"""
    assert k <= len(r)
    assert k != 0
    return sum(int(x) for x in r[:k]) / k

def MAP_at_3(predictions, true_items):
    """Score is mean average precision at 3"""
    U = len(predictions)
    map_at_3 = 0.0
    for u in range(U):
        user_preds = predictions[u].split()
        user_true = true_items[u]
        user_results = [1 if item == user_true else 0 for item in user_preds]
        for k in range(min(len(user_preds), 3)):
            map_at_3 += precision_at_k(user_results, k+1) * user_results[k]
    return map_at_3 / U

# Train and Save 
We now train our model using the Hugging Face Trainer API and leverage Weights & Biases (W&B) Sweeps to perform hyperparameter search. A great article that describes this method can be found [here][1].

[1]: https://wandb.ai/matt24/vit-snacks-sweeps/reports/Hyperparameter-Search-for-HuggingFace-Transformer-Models--VmlldzoyMTUxNTg0

In [13]:
def train(config=None):
    with wandb.init(config=config):
        # set sweep configuration
        config = wandb.config
        
        # Run the wandb magic comand!
        # This displays the details of each training
        %wandb

        # set training arguments
        training_args = TrainingArguments(
            output_dir = f'/kaggle/working/checkpoints_{VER}',
            report_to='wandb',
            num_train_epochs=config.epochs,
            learning_rate=config.learning_rate,
            weight_decay=config.weight_decay,
            warmup_ratio=config.warmup_ratio,
            gradient_accumulation_steps=config.gradient_accumulation_steps,
            per_device_train_batch_size=1,
            per_device_eval_batch_size=2,
            overwrite_output_dir=True,
            fp16=True,
            logging_steps=25,
            evaluation_strategy='steps',
            eval_steps=25,
            save_strategy="steps",
            save_steps=25,
            load_best_model_at_end=True,
            metric_for_best_model='map@3',
            lr_scheduler_type='cosine',
            save_total_limit=2,
        )

        # define training loop
        trainer = Trainer(
            model_init=model_init,
            args=training_args,
            tokenizer=tokenizer,
            data_collator=DataCollatorForMultipleChoice(tokenizer=tokenizer),
            train_dataset=tokenized_train_dataset,
            eval_dataset=tokenized_val_dataset,
            compute_metrics = compute_metrics,
            callbacks=[EarlyStoppingCallback(early_stopping_patience=config.early_stopping_patience)],
        )

        # start training loop
        trainer.train()
        
        # Save the trained model
        trainer.save_model(f'/kaggle/working/model_v{VER}')
        
        
        model = AutoModelForMultipleChoice.from_pretrained(f'/kaggle/working/model_v{VER}')
        trainer = Trainer(model=model,
                          tokenizer=tokenizer,
                          data_collator=DataCollatorForMultipleChoice(tokenizer=tokenizer))
        
        # Verify Saved Model
        test_predictions = trainer.predict(tokenized_test_dataset).predictions
        predictions_as_ids = np.argsort(-test_predictions, 1)
        predictions_as_answer_letters = np.array(list('ABCDE'))[predictions_as_ids]
        predictions_as_string = test_df['prediction'] = [
            ' '.join(row) for row in predictions_as_answer_letters[:, :3]
        ]
        test_labels = [option_to_index[answer] for answer in tokenized_test_dataset['answer']]
        
        m = MAP_at_3(test_df.prediction.values, test_df.answer.values)
        test_accuracy = accuracy_score(test_labels, test_predictions.argmax(axis=1))
        
        # Log the metrics on the test set
        wandb.log({'Test MAP@3': m,
                  'Test Accuracy': test_accuracy})
        
        print('Test MAP@3 = ',m)
        print('Test Accuracy = ', test_accuracy)

In [14]:
# Run the sweep agent
wandb.agent(sweep_id, train, count=3)

wandb: Agent Starting Run: yajwym7u with config:
wandb: 	early_stopping_patience: 5
wandb: 	epochs: 30
wandb: 	gradient_accumulation_steps: 2
wandb: 	learning_rate: 6.872656905040427e-05
wandb: 	warmup_ratio: 0.2
wandb: 	weight_decay: 0.01
wandb: Currently logged in as: emmermarci (import_this). Use `wandb login --relogin` to force relogin
cat: /sys/module/amdgpu/initstate: No such file or directory


Some weights of DebertaV2ForMultipleChoice were not initialized from the model checkpoint at microsoft/deberta-v3-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight', 'classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Freezing embeddings.
Freezing 18 layers.


Some weights of DebertaV2ForMultipleChoice were not initialized from the model checkpoint at microsoft/deberta-v3-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight', 'classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Freezing embeddings.
Freezing 18 layers.


...

wandb: WARNING Config item 'gradient_accumulation_steps' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'learning_rate' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'weight_decay' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'warmup_ratio' was locked by 'sweep' (ignored update).
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss,Validation Loss,Map@3,Accuracy
25,1.509700,1.098663,0.740000,0.600000
50,0.330400,1.303282,0.753333,0.640000
75,0.036900,2.119763,0.710000,0.560000
100,0.010200,3.666858,0.686667,0.500000
125,0.004500,1.945296,0.733333,0.540000
150,0.000200,2.311602,0.750000,0.580000


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead u

Test MAP@3 =  0.7366666666666667
Test Accuracy =  0.58


Test Accuracy,▁
Test MAP@3,▁
eval/accuracy,▆█▄▁▃▅
eval/loss,▁▂▄█▃▄
eval/map@3,▇█▃▁▆█
eval/runtime,▃▁▂█▂▂
eval/samples_per_second,▆█▇▁▇▇
eval/steps_per_second,▆█▇▁▇▇
train/epoch,▁▁▂▂▄▄▅▅▇▇███
train/global_step,▁▁▂▂▄▄▅▅▇▇████
train/learning_rate,█▇▅▃▂▁


We are far behind the best MAP@3 metric of the competition, which is 0.933208 made by [Team H2O LLM Studio][1] but that is to be expected. They have used a dataset of 2.46 TB of data, where we only used our original 200 questions and answers with the additional context column. Nonetheless, it is exciting to see that our model reached a far better outcome than random guessing from such a small dataset.


MAP@3 is the official evaluaion metric of the competition, but out of curiosity we checked the accuracy too.

[1]: https://www.kaggle.com/competitions/kaggle-llm-science-exam/discussion/446422